In [1]:
from tools import tools
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer, util
from torch import device, cuda
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model_name = 'all-mpnet-base-v2'
model = SentenceTransformer(model_name)

model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [4]:
reports_file_path = 'data/br/firefox/firefox.csv'
relations_file_path = 'data/br/firefox/firefox_pairs.csv'

reports_firefox = pd.read_csv(reports_file_path, index_col='bug_id')
relations_firefox = pd.read_csv(relations_file_path)

In [5]:
test_set = reports_firefox[0:5]
test_set.head()

,priority,component,dup_id,short_desc,description,bug_status,resolution,version,creation_ts,delta_ts
bug_id,,,,,,,,,,
10954,P3,Preferences,[],Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700
14871,--,General,269442,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700
19118,--,Preferences,[],Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800
54746,P3,Preferences,[],Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800
56892,P3,General,[],Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700


In [6]:
reports_firefox = tools.generate_encodings(reports_firefox, model_name)
reports_firefox[model_name].to_csv('firefox_mpnet_embeddings.csv')

In [7]:
embeddings_file_path = 'firefox_mpnet_embeddings.csv'

embeddings_firefox = pd.read_csv(embeddings_file_path, index_col='bug_id')

In [8]:
embeddings_firefox.head()

,all-mpnet-base-v2
bug_id,
10954,[-1.12741916e-02 9.21382606e-02 -1.07803373e-...
14871,[ 2.88254395e-02 4.96729985e-02 -2.00148881e-...
19118,[ 6.70061773e-03 -4.01917240e-03 -3.18142958e-...
54746,[ 1.39751136e-02 -2.73819230e-02 1.22563411e-...
56892,[ 2.35558790e-03 3.97690348e-02 9.72936919e-...


In [9]:
embeddings_firefox[embeddings_firefox['all-mpnet-base-v2'] == None]

,all-mpnet-base-v2
bug_id,


In [10]:
len(embeddings_firefox)

115814